<a href="https://colab.research.google.com/github/anonyblank/ml-zoomcamp-hw/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!wget 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'

--2023-09-29 11:37:22--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv.10’

data.csv.10         100%[===================>]   1.41M  6.11MB/s    in 0.2s    

2023-09-29 11:37:23 (6.11 MB/s) - ‘data.csv.10’ saved [1475504/1475504]



In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
raw_datasets = pd.read_csv('data.csv')

In [46]:
selected_cols = [
    'Make',
    'Model',
    'Year',
    'Engine HP',
    'Engine Cylinders',
    'Transmission Type',
    'Vehicle Style',
    'highway MPG',
    'city mpg',
    'MSRP'
]

In [47]:
datasets = raw_datasets[selected_cols]

In [48]:
datasets.columns = datasets.columns.str.replace(' ', '_').str.lower()

In [49]:
datasets = datasets.fillna(0)

In [50]:
datasets = datasets.rename(columns={"msrp": "price"})

In [51]:
datasets

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500
...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,46120
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,56670
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50620
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50920


In [52]:
datasets.transmission_type.mode()

0    AUTOMATIC
Name: transmission_type, dtype: object

In [53]:
numerical = [
    'year',
    'engine_hp',
    'engine_cylinders',
    'highway_mpg',
    'city_mpg'
]

In [54]:
# Bulid in correlation_matrix
correlation_matrix = datasets[numerical].corr()
correlation_matrix

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg
year,1.000000,0.338714,-0.040708,0.258240,0.198171
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000


In [55]:
# Own correlation_matrix
corr_matrix = pd.DataFrame(index=numerical)

for numeric in numerical:
  corr_matrix[numeric] = datasets[numerical].corrwith(datasets[numeric]).values

corr_matrix

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg
year,1.000000,0.338714,-0.040708,0.258240,0.198171
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000


In [56]:
biggest_correlation = correlation_matrix.unstack().sort_values(ascending=False)[:15]
biggest_correlation

year              year                1.000000
engine_hp         engine_hp           1.000000
highway_mpg       highway_mpg         1.000000
engine_cylinders  engine_cylinders    1.000000
city_mpg          city_mpg            1.000000
                  highway_mpg         0.886829
highway_mpg       city_mpg            0.886829
engine_hp         engine_cylinders    0.774851
engine_cylinders  engine_hp           0.774851
engine_hp         year                0.338714
year              engine_hp           0.338714
                  highway_mpg         0.258240
highway_mpg       year                0.258240
city_mpg          year                0.198171
year              city_mpg            0.198171
dtype: float64

In [57]:
datasets["average_above"] = (datasets.price > datasets.price.mean()).astype(int).values

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
datasets_for_full_train, datasets_for_test = train_test_split(datasets, test_size=0.2, random_state=42)

In [60]:
datasets_for_train, datasets_for_val = train_test_split(datasets_for_full_train, test_size=0.25, random_state=42)

In [61]:
len(datasets_for_train + datasets_for_val + datasets_for_test), len(datasets)

(11914, 11914)

In [62]:
datasets_for_full_train = datasets_for_full_train.reset_index(drop=True)
datasets_for_train = datasets_for_train.reset_index(drop=True)
datasets_for_val = datasets_for_val.reset_index(drop=True)
datasets_for_test = datasets_for_test.reset_index(drop=True)
datasets_for_test

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,average_above
0,GMC,Envoy XL,2005,275.0,6.0,AUTOMATIC,4dr SUV,18,13,29695,0
1,Volkswagen,Passat,2016,170.0,4.0,AUTOMATIC,Sedan,38,25,30495,0
2,Honda,Odyssey,2016,248.0,6.0,AUTOMATIC,Passenger Minivan,28,19,37650,0
3,Chevrolet,Cruze,2015,138.0,4.0,MANUAL,Sedan,36,25,16170,0
4,Volvo,740,1991,162.0,4.0,AUTOMATIC,Sedan,20,17,2000,0
...,...,...,...,...,...,...,...,...,...,...,...
2378,Nissan,350Z,2007,306.0,6.0,MANUAL,Convertible,24,17,35550,0
2379,Ford,Expedition,2016,365.0,6.0,AUTOMATIC,4dr SUV,20,15,48360,1
2380,Acura,ILX,2015,150.0,4.0,AUTOMATIC,Sedan,35,24,31750,0
2381,Volkswagen,Jetta SportWagen,2014,170.0,5.0,MANUAL,Wagon,33,23,20995,0


In [63]:
categorical = [
    'make',
    'model',
    'transmission_type',
    'vehicle_style'
]

In [64]:
from sklearn.metrics import mutual_info_score

In [65]:
def mutual_info_average_score(series):
  return mutual_info_score(datasets_for_train.average_above, series)

score = datasets_for_train[categorical].apply(mutual_info_average_score).sort_values(ascending=True)
round(score, 2)

transmission_type    0.02
vehicle_style        0.08
make                 0.24
model                0.46
dtype: float64

In [66]:
from sklearn.feature_extraction import DictVectorizer

In [67]:
dv = DictVectorizer(sparse=False)

dict_datasets_for_train = datasets_for_train[categorical+numerical].to_dict(orient='records')
dv.fit(dict_datasets_for_train)
features_for_train = dv.transform(dict_datasets_for_train)

dict_datasets_for_val = datasets_for_val[categorical+numerical].to_dict(orient='records')
features_for_val = dv.transform(dict_datasets_for_val)

In [68]:
labels_for_train = datasets_for_train['average_above'].values
labels_for_train

array([0, 0, 1, ..., 0, 0, 0])

In [69]:
from sklearn.linear_model import LogisticRegression

In [70]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(features_for_train,labels_for_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [71]:
pred_labels_for_val = model.predict(features_for_val)
pred_labels_for_val

array([0, 1, 0, ..., 0, 1, 1])

In [72]:
labels_for_val = datasets_for_val.average_above.values

In [73]:
from sklearn.metrics import accuracy_score
accuracy = np.round(accuracy_score(labels_for_val, pred_labels_for_val),2)
accuracy

0.93

In [74]:
features_list = [
    'year',
    'engine_hp',
    'transmission_type',
    'city_mpg'
]

dv = DictVectorizer(sparse=False)

dict_datasets_for_train = datasets_for_train[features_list].to_dict(orient='records')
dv.fit(dict_datasets_for_train)
features_for_train = dv.transform(dict_datasets_for_train)

dict_datasets_for_val = datasets_for_val[features_list].to_dict(orient='records')
features_for_val = dv.transform(dict_datasets_for_val)

labels_for_train = datasets_for_train.average_above.values
labels_for_val = datasets_for_val.average_above.values

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(features_for_train, labels_for_train)

pred_labels_for_val = model.predict(features_for_val)
score = accuracy_score(labels_for_val, pred_labels_for_val)
score

0.8850188837599664

In [75]:
org_score = score
for feature in features_list:

  features = features_list.copy()
  features.remove(feature)

  dv = DictVectorizer(sparse=False)

  dict_datasets_for_train = datasets_for_train[features].to_dict(orient='records')
  dv.fit(dict_datasets_for_train)
  features_for_train = dv.transform(dict_datasets_for_train)

  dict_datasets_for_val = datasets_for_val[features].to_dict(orient='records')
  features_for_val = dv.transform(dict_datasets_for_val)

  model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
  model.fit(features_for_train, labels_for_train)

  pred_labels_for_val = model.predict(features_for_val)
  score = accuracy_score(labels_for_val, pred_labels_for_val)
  print(feature, org_score - score, org_score, score)

year -0.00041963911036513313 0.8850188837599664 0.8854385228703315
engine_hp 0.14057910197230383 0.8850188837599664 0.7444397817876626
transmission_type 0.002937473772555599 0.8850188837599664 0.8820814099874108
city_mpg 0.008392782207301663 0.8850188837599664 0.8766261015526647


In [76]:
# year -0.013848090642047839 0.9483843894250944 0.9345362987830466
# engine_hp -0.00041963911036513313 0.9349559378934117 0.9345362987830466
# transmission_type -0.010490977759127218 0.9450272765421738 0.9345362987830466
# city_mpg -0.011330255979857373 0.9458665547629039 0.9345362987830466

In [77]:
price_labels_for_train = np.log1p(datasets_for_train.price.values)
price_labels_for_val = np.log1p(datasets_for_val.price.values)
price_labels_for_test = np.log1p(datasets_for_test.price.values)

In [78]:
price_labels_for_test

array([10.29876764, 10.32535081, 10.53611479, ..., 10.3656795 ,
        9.95208722, 10.96302978])

In [79]:
# from sklearn.linear_model import Ridge
# from sklearn.metrics import mean_squared_error

# alpha_list = [0, 0.01, 0.1, 1, 10]

# dv = DictVectorizer(sparse=False)

# dict_datasets_for_train = datasets_for_train[categorical+numerical].to_dict(orient='records')
# dv.fit(dict_datasets_for_train)
# features_for_train = dv.transform(dict_datasets_for_train)

# dict_datasets_for_val = datasets_for_val[categorical+numerical].to_dict(orient='records')
# features_for_val = dv.transform(dict_datasets_for_val)


# for alpha in alpha_list:
#   model_r = Ridge(alpha=alpha, solver='sag', random_state=42)
#   model_r.fit(features_for_train, price_labels_for_train)

#   price_pred_labels_for_val = model_r.predict(features_for_val)

#   rmse = mean_squared_error(price_labels_for_val, price_pred_labels_for_val, squared=False)

#   print(rmse)
# # 0.4867943132423885
# # 0.4867945519275268
# # 0.48679670001899794
# # 0.4868181745432731
# # 0.48703228329751225